<a href="https://colab.research.google.com/github/AyushDhimann/Stable-Diffusion-Linux/blob/main/RunStableDiffusionOnColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Prerequisite

!sudo apt install libtcmalloc-minimal4 libegl1 libgl1-mesa-glx -y

In [ ]:
# Current working directory


import os
import requests
from urllib.parse import urlparse, parse_qs, unquote
import re
%pwd

In [ ]:
# Installation


%mkdir StableDiffusionHere
%cd {os.getcwd()}/StableDiffusionHere

!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui
%cd {os.getcwd()}/stable-diffusion-webui

In [ ]:
# Download Models

MODEL_LINK = ""  # Add the model link here

In [ ]:
# Downloader

def get_info(url):
    parsed_url = urlparse(url)
    src = 'civitai' if parsed_url.netloc == 'civitai.com' else 'huggingface' if parsed_url.netloc == 'huggingface.co' else 'others'
    response = requests.get(url, allow_redirects=False)
    redirected_url = response.headers.get("Location", "")
    quer = parse_qs(urlparse(redirected_url).query) if "response-content-disposition" in (quer := parse_qs(urlparse(redirected_url).query)) else {}
    filenm = unquote(quer.get("response-content-disposition", [""])[0].split("=", 1)[1].strip()).replace("\"", "") if quer and quer.get("response-content-disposition") else ""
    return src, filenm

src, filename = get_info(MODEL_LINK)

In [ ]:
#Download Stable-Diffusion Models

!wget -O "{os.getcwd()}/models/Stable-diffusion/{filename}" "{MODEL_LINK}"

In [ ]:
#Download other Models


model_dir = "" # Add the chosen model type here


if not os.path.exists('{os.getcwd()}/models/{model_dir}'):
    print(f"Directory '{model_dir}' not found.")
    print("List of total directories:")
    %ls {os.getcwd()}/models/
else:
    # If the directory exists, proceed with wget
    !wget -O "{os.getcwd()}/models/{model_dir}/{filename}" "{MODEL_LINK}

In [ ]:
#Minor File Changes(removes the root user restriction and disables the need of venv)

for _ in range(2):
    file_path = f"{os.getcwd()}/webui.sh"
    with open(file_path, "r+") as file:
        replacements = {'use_venv': ('1', '0'), 'can_run_as_root': ('0', '1')}
        content = re.sub('|'.join([f'{key}={old}' for key, (old, new) in replacements.items()]), lambda m, replacements=replacements: f'{m.group(0).split("=")[0]}={replacements[m.group(0).split("=")[0]][1]}', file.read(), count=1)
        file.seek(0)
        file.write(content)
        file.truncate()

In [ ]:
#Run WebUI (only CPU)[Username = s : Password = d]

! ./webui.sh --share --gradio-auth s:d --administrator --no-half --use-cpu all --opt-sdp-no-mem-attention --no-half-vae --opt-channelslast --skip-torch-cuda-test

In [ ]:
#Run WebUI (with CPU+GPU)[Username = s : Password = d]

! ./webui.sh --share --gradio-auth s:d --no-half --opt-sdp-no-mem-attention --no-half-vae --opt-channelslast --xformers